# Imports 

In [3]:
## Essential Imports: 
import os
import numpy as np
import qp
import tables_io
from pathlib import Path 
from pzflow.examples import get_galaxy_data
import ceci

## RAIL-Specific Imports: 
import rail

# old : from rail.creation.degradation import LSSTErrorModel, InvRedshiftIncompleteness


from rail.creation.degradation.lsst_error_model import LSSTErrorModel
from rail.creation.degradation.spectroscopic_degraders import InvRedshiftIncompleteness

import rail.creation 
import rail.creation.engines
from rail.creation.engines.flowEngine import FlowModeler, FlowCreator, FlowPosterior
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

# old : from rail.estimation.algos.flexzboost import Inform_FZBoost, FZBoost

from rail.estimation.algos.train_z import TrainZEstimator, TrainZInformer
from rail.estimation.algos.cmnn import Inform_CMNNPDF, CMNNPDF
# from rail.estimation.algos.gpz import GPzInformer, GPzEstimator 
from rail.estimation.algos.pzflow_nf import PZFlowInformer, PZFlowEstimator 
#from rail.estimation.algos.flexzboost import FlexZBoostInformer, FlexZBoostEstimator  

# from rail.estimation.algos.k_nearneigh import #Inform_KNearNeighPDF, Inform_KNearNeighPDF 
#from rail.estimation.algos.minisom_som import MiniSOMInformer, MiniSOMEstimator 
#from rail.estimation.algos.sklearn_neurnet import #Inform_SimpleNN, Inform_SimpleNN 
#from rail.estimation.algos.somoclu_som import SOMocluInformer, SOMocluInformer

#from rail.estimation.algos.bpz_lite import BPZliteInformer, BPZliteEstimator


from rail.evaluation.evaluator import Evaluator


## Data Storage: 
DS = RailStage.data_store
DS.__class__.allow_overwrite = True


### CMNN, PZFlow, FlexZBoost, GPZ, trainz for control

In [44]:
 # help(rail.estimation.algos.cmnn)

In [3]:
# help(rail.creation.engines)

In [3]:
#from rail.stages import *
#rail.stages.import_and_attach_all()
#for val in RailStage.pipeline_stages.values():
#    print(val[0])

# Model

In [4]:
def makeModel():
    #path to access the data 
    DATA_DIR = Path().resolve() / "data"
    DATA_DIR.mkdir(exist_ok=True)

    catalog_file = DATA_DIR / "base_catalog.pq"

    bands = ['u','g','r','i','z','y']
    band_dict = {band:f'mag_{band}_lsst' for band in bands}
    
    #array of galaxies w/ 7 attributes for each: redshift & ugrizy
    catalog = get_galaxy_data().rename(band_dict, axis=1) 

    #turns array into a table 
    tables_io.write(catalog, str(catalog_file.with_suffix("")), catalog_file.suffix[1:])

    catalog_file = str(catalog_file)
    flow_file = str(DATA_DIR / "trained_flow.pkl")

    print(flow_file)

    #we set up the stage 
    flow_modeler_params = {
        "name": "flow_modeler",
        "input": catalog_file,
        "model": flow_file,
        "seed": 0,
        "phys_cols": {"redshift": [0, 3]},
        "phot_cols": {
            "mag_u_lsst": [17, 35],
            "mag_g_lsst": [16, 32],
            "mag_r_lsst": [15, 30],
            "mag_i_lsst": [15, 30],
            "mag_z_lsst": [14, 29],
            "mag_y_lsst": [14, 28],
        },
        "calc_colors": {"ref_column_name": "mag_i_lsst"},
    }
    flow_modeler = FlowModeler.make_stage(**flow_modeler_params)
    # flow_modeler.fit_model()
    return flow_modeler, flow_file ##.get_handle("model")

In [5]:
modelData, flow_file = makeModel() 

/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl


# Make Training Set and Test Set 

In [6]:
def trainSet(ntrain, seed):
    data = FlowCreator.make_stage(
            name = 'train_set',
            model = flow_file,
            n_samples = ntrain,
            seed = seed 
    )
    return data 

In [7]:
def testSet(ntest, seed):
    data = FlowCreator.make_stage(
            name = 'test_set',
            model = flow_file,
            n_samples = ntest,
            seed = seed 
    )
    return data #.sample(ntest, seed)

# Degraders

## Inverse Redshift Incompleteness

In [8]:
def invRedshift(pivot = 1.0):
    assert type(pivot) == float 
    degr = InvRedshiftIncompleteness.make_stage(
        name = 'inv_redshift',
        pivot_redshift = pivot
    )
    return degr 

pivot_ls = [1.0, 1.4]

In [9]:
# ## Choose pivot z's for inverse redshift incompleteness 

# ## seed1 and ndata should be the same as  seed1 and ntrain used to call bigF!! 
# ## Otherwise this might not be representative of the real data 

# def choosePivots(seed1, ndata):
#     nums = trainSet(ndata, seed1)
#     data = nums.sample(ndata, seed1)
#     data_pq = col_remap(data)
#     data_table = table_conv(data_pq)
#     table = tables_io.convertObj(data_table.data, tables_io.types.PD_DATAFRAME)
#     return np.asarray(table['redshift'])

# percentiles = np.arange(10, 100, 10)
# pivots = [] 

# for i in percentiles:
#     pivot = np.percentile(choosePivots(17, 100000), i) 
#     pivots.append(pivot)

In [10]:
# print(pivots)

## LSST Error 

In [11]:
bands = ['u','g','r','i','z','y']
band_dict = {band:f'mag_{band}_lsst' for band in bands}

def lsstError(dict, seed): #tvis = 1, nYrObs = 1, airmass = 1, extendedSource = 1, sigmaSys = 1, magLim = 1, ndFlag = 1, A_min = 1, A_max = 1):
    deg = LSSTErrorModel.make_stage(
        name='lsst_error',
        renameDict= dict, 
        ndFlag=np.nan,
        seed=seed,
    )
    return deg 

## Quantity Cuts 

In [12]:
## write a dictionary with the different bands and magnitudes you want

def quantCuts(band, mag):
    quantity_cut = QuantityCut.make_stage(
        name='quantity_cut',    
        cuts={'mag_i_lsst': 25.0},
    )

In [13]:
qcuts_dict = {'mag_u_lsst': [...], 
              'mag_g_lsst': [...], 
              'mag_r_lsst': [...], 
              'mag_i_lsst': [...], 
              'mag_z_lsst': [...], 
              'mag_y_lsst': [...] }

## Survey-Based Degraders

In [14]:
from rail.creation.degradation.spectroscopic_selections import *

def specSelectBOSS(ntrain):
    degr = SpecSelection_BOSS.make_stage(
        name = 'specselection_boss',
        N_tot = ntrain
    )
    return degr 

def specSelectDEEP2(ntrain):
    degr = SpecSelection_DEEP2.make_stage(
        name = 'specselection_deep2',
        N_tot = ntrain
    )
    return degr 

def specSelectGAMA(ntrain):
    degr = SpecSelection_GAMA.make_stage(
        name = 'specselection_gama',
        N_tot = ntrain
    )
    return degr 

def specSelectHSC(ntrain):
    degr = SpecSelection_HSC.make_stage(
        name = 'specselection_HSC',
        N_tot = ntrain
    )
    return degr 

def specSelectVVDSf02(ntrain):
    degr = SpecSelection_VVDSf02.make_stage(
        name = 'specselection_VVDSf02',
        N_tot = ntrain
    )
    return degr 

def specSelectzCOSMOS(ntrain):
    degr = SpecSelection_zCOSMOS.make_stage(
        name = 'specselection_zCOSMOS',
        N_tot = ntrain
    )
    return degr 

In [15]:
spec_dict = {'BOSS': specSelectBOSS, 
             'DEEP2': specSelectDEEP2, 
             'GAMA': specSelectGAMA,
             'HSC': specSelectHSC, 
             'VVDSf02': specSelectVVDSf02, 
             'zCOSMOS': specSelectzCOSMOS } 

## Posts 

In [16]:
def getPosts(data, model, grid):
    posts = FlowPosterior.make_stage(
        name='get_posts'+str(data), 
        column='redshift',
        grid = grid,
        model = model,
        data = data
    )
    return posts #posts.get_posterior(data, column = 'redshift')

In [17]:
def makeGrid(zmin, zmax, nbins):
    import numpy as np
    grid = np.linspace(zmin, zmax, nbins + 1)
    return grid 

In [18]:
grid = makeGrid(0, 2.5, 100)

Only run if you need output_orig_train_posts

In [19]:
# flow_post_orig_train = FlowPosterior.make_stage(name='orig_train_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = orig_train)

# orig_train_pdfs = flow_post_orig_train.get_posterior(orig_train, column='redshift')

Only run if you need output_deg_train_posts ** rerun this cell!! 

In [20]:
# flow_post_deg_train = FlowPosterior.make_stage(name='deg_train_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              err_samples = 0,
#                                              data = deg_train)



# deg_train_pdfs = flow_post_deg_train.get_posterior(deg_train, column='redshift')

Only run if you need output_orig_test_posts

In [21]:
# flow_post_orig_test = FlowPosterior.make_stage(name='orig_test_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = orig_test)

# orig_test_pdfs = flow_post_orig_test.get_posterior(orig_test, column='redshift')

Only run if you need output_deg_test_posts

In [22]:
# flow_post_deg_test = FlowPosterior.make_stage(name='deg_test_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = deg_test)

# deg_test_pdfs = flow_post_deg_test.get_posterior(deg_test, column='redshift')

# Make tables

In [23]:
bands = ['u','g','r','i','z','y']
band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

def colRemapper(dict):
    col_remap = ColumnMapper.make_stage(
    name='col_remapper', 
    columns=dict,
    )
    return col_remap

def tableConverter():
    table_conv = TableConverter.make_stage(
    name='table_conv', 
    output_format='numpyDict',
    )
    return table_conv

In [24]:
col_remap = colRemapper(band_dict_err)
table_conv = tableConverter()

# Inform & Estimate

In [25]:
def informTrainZ():
    inf = TrainZInformer.make_stage(
    name = 'inform_TrainZ',
    model = 'trainz.pkl',
    hdf5_groupname=""
    )
    return inf

def estimateTrainZ(info):
    est = TrainZEstimator.make_stage(
    name = 'estimate_TrainZ',
    model = 'trainz.pkl', 
    hdf5_groupname=""
    )
    return est

In [26]:
def informCMNN():
    inf = Inform_CMNNPDF.make_stage(
    name = 'inform_CMNN',
    model = 'cmnn.pkl',
    hdf5_groupname=""
    )
    return inf

def estimateCMNN(info):
    est = CMNNPDF.make_stage(
    name = 'estimate_CMNN',
    model = 'cmnn.pkl', 
    hdf5_groupname=""
    )
    return est

In [27]:
def informGPz():
    inf = GPzInformer.make_stage(
    name = 'inform_GPz',
    model = 'gpz.pkl',
    hdf5_groupname=""
    )
    return inf

def estimateGPz(info):
    est = GPzEstimator.make_stage(
    name = 'estimate_GPz',
    model = 'gpz.pkl', 
    hdf5_groupname=""
    )
    return est

In [28]:
def informPZFlow():
    inf = PZFlowInformer.make_stage(
    name = 'inform_PZFlow',
    model = 'pzflow.pkl',
    hdf5_groupname=""
    )
    return inf

def estimatePZFlow(info):
    est = PZFlowEstimator.make_stage(
    name = 'estimate_PZFlow',
    model = 'pzflow.pkl', 
    hdf5_groupname=""
    )
    return est

In [29]:
def informFZBoost():
    info = FlexZBoostInformer.make_stage(
    name ='inform_FZBoost', 
    model ='fzboost.pkl', 
    hdf5_groupname='',
    )
    return info

def estimateFZBoost(info):
    est = FlexZBoostEstimator.make_stage(
    name='est_FZBoost', 
    nondetect_val=np.nan,
    model= info,
    hdf5_groupname='',
    aliases=dict(input='test_data', output='fzboost_estim'),
    nzbins = 100 
    )
    return est 

In [30]:
inf_est_dict = {'TrainZ': [informTrainZ, estimateTrainZ],
               'CMNN': [informCMNN, estimateCMNN], 
               'GPz': [informGPz, estimateGPz], 
               'PZFlow': [informPZFlow, estimatePZFlow], 
               'FZBoost': [informFZBoost, estimateFZBoost]}

# Big F

In [35]:
def bigF(degrader, degname, informer, estimator, estname, ntrain, ntest, seed1, seed2, seed3):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    
    ## have to comment out the (ntrain) when running with inverse redshift incompleteness
    deg = degrader #(ntrain) 
    
    testData = testSet(ntest, seed2)
    lsstErr = lsstError(band_dict, seed3)
    
    inf = informer()
    est = estimator(inf)


    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        #deg, 
        testData, 
        lsstErr,  
        inf, 
        est ] 

    for stage in stages:
        pipe.add_stage(stage)
    
    ## have to comment out deg.connect_input to run control case 
    
    # deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    inf.connect_input(trainData)#(deg) 
    est.connect_input(inf, inputTag = 'model')
    est.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    directory = str(degname) + "_" + str(estname)
    parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"#"/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs"
    path = os.path.join(parent_dir, directory)
    os.makedirs(path)
    
    outpath = os.path.join(path, "% s.yml" % str(degname + "_" + estname))
    pipe.save(outpath)
    return outpath 

In [36]:
# can alice do strings? 

word = "bestest"
word2 = "cat"


print("%s_ever" % str(word +"_"+ word2))

bestest_cat_ever


In [37]:
# 'invz': invRedshift,

spec_dict = {'BOSS': specSelectBOSS, 
             'DEEP2': specSelectDEEP2, 
             'GAMA': specSelectGAMA,
             'HSC': specSelectHSC, 
             'VVDSf02': specSelectVVDSf02, 
             'zCOSMOS': specSelectzCOSMOS } 

inf_est_dict = {'TrainZ': [informTrainZ, estimateTrainZ],
               'CMNN': [informCMNN, estimateCMNN], 
               'GPz': [informGPz, estimateGPz], 
               'PZFlow': [informPZFlow, estimatePZFlow], 
               'FZBoost': [informFZBoost, estimateFZBoost] }

In [38]:
##run 

path_ls = []

for key1 in spec_dict:
    for key2 in inf_est_dict:
        path_ls.append(bigF(spec_dict[key1], key1, inf_est_dict[key2][0], inf_est_dict[key2][1], key2, 100000, 100000, 17, 39, 172))

Inserting handle into data store.  model_inform_CMNN: inprogress_cmnn.pkl, inform_CMNN


NameError: name 'GPzInformer' is not defined

In [ ]:
for key2 in inf_est_dict:
    path_ls.append(bigF(invRedshift(1.0), "invz=1.0", inf_est_dict[key2][0], inf_est_dict[key2][1], key2, 100000, 100000, 17, 39, 172))
    path_ls.append(bigF(invRedshift(1.4), "invz=1.4", inf_est_dict[key2][0], inf_est_dict[key2][1], key2, 100000, 100000, 17, 39, 172))

Inserting handle into data store.  output_inv_redshift: inprogress_output_inv_redshift.pq, inv_redshift
Failed to save {'train_set': {'output_mode': 'default', 'n_samples': 100000, 'seed': 17, 'name': 'train_set', 'model': '/global/u2/a/acraffor/Photo-z-Stress-Test/data/trained_flow.pkl', 'config': None, 'aliases': {'output': 'output_train_set'}}, 'inv_redshift': {'output_mode': 'default', 'seed': 12345, 'pivot_redshift': 1.0, 'name': 'inv_redshift', 'config': None, 'input': 'None', 'aliases': {'output': 'output_inv_redshift', 'input': 'output_train_set'}}, 'test_set': {'output_mode': 'default', 'n_samples': 100000, 'seed': 39, 'name': 'test_set', 'model': '/global/u2/a/acraffor/Photo-z-Stress-Test/data/trained_flow.pkl', 'config': None, 'aliases': {'output': 'output_test_set'}}, 'lsst_error': {'output_mode': 'default', 'seed': 172, 'nYrObs': 10.0, 'nVisYr': {'u': 5.6, 'g': 8.0, 'r': 18.4, 'i': 18.4, 'z': 16.0, 'y': 16.0}, 'gamma': {'u': 0.038, 'g': 0.039, 'r': 0.039, 'i': 0.039, 'z': 

In [ ]:

for key2 in inf_est_dict:
    path_ls.append(bigF(None, "undegraded", inf_est_dict[key2][0], inf_est_dict[key2][1], key2, 100000, 100000, 17, 39, 172))
    

Inserting handle into data store.  output_test_set: inprogress_output_test_set.pq, test_set
Inserting handle into data store.  output_train_set: inprogress_output_train_set.pq, train_set
Inserting handle into data store.  model_inform_TrainZ: inprogress_trainz.pkl, inform_TrainZ
Inserting handle into data store.  output_lsst_error: inprogress_output_lsst_error.pq, lsst_error
Inserting handle into data store.  model_inform_CMNN: inprogress_cmnn.pkl, inform_CMNN
Inserting handle into data store.  model_inform_GPz: inprogress_gpz.pkl, inform_GPz
Inserting handle into data store.  model_inform_PZFlow: inprogress_pzflow.pkl, inform_PZFlow
Inserting handle into data store.  model_inform_FZBoost: inprogress_fzboost.pkl, inform_FZBoost
Failed to save {'train_set': {'output_mode': 'default', 'n_samples': 100000, 'seed': 17, 'name': 'train_set', 'model': '/global/u2/a/acraffor/Photo-z-Stress-Test/data/trained_flow.pkl', 'config': None, 'aliases': {'output': 'output_train_set'}}, 'test_set': {'ou

# Big F 1.0

In [ ]:
def bigF1(pivotz, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    ##stages 
    trainData = trainSet(ntrain, seed1)
    invRed = invRedshift(float(pivotz))

    # origTrainPosts = getPosts(output_train_set.pq (???), modelData, grid)
    # degTrainPosts = getPosts(###)

    testData = testSet(ntest, seed2)
    lsstErr = lsstError(band_dict, seed3)

    # origTestPosts = getPosts(###)
    # degTestPosts = getPosts(###)

    # informFZB = informFZBoost()
    # estFZB = estimateFZBoost(informFZB, nbins)

    infPZFlow = informPZFlow()
    estPZFlow = estimatePZFlow(infPZFlow)

    
    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        invRed, 
        testData, 
        lsstErr,  
        infPZFlow, 
        estPZFlow]
        #informFZB, 
        #estFZB]
    
    for stage in stages:
        pipe.add_stage(stage)
        

    invRed.connect_input(trainData)
    lsstErr.connect_input(testData)

    infPZFlow.connect_input(invRed) 
    estPZFlow.connect_input(infPZFlow, inputTag = 'model')
    estPZFlow.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    # informFZB.connect_input(invRed)
    # estFZB.connect_input(informFZB, lsstErr) 
    
    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(path_1, "invz='% s'_lsstErr_pzflow.yml" % %.3f%(pivotz))
    pipe.save(outpath)
    return outpath 
   

## Run Big F 1.0

In [ ]:
path_lst_1 = []
directory = "invz_lsstErr_pzflow"
parent_dir = "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs"
path_1 = os.path.join(parent_dir, directory)
os.makedirs(path_1)

for i in (pivots):
    path_lst_1.append(bigF1(i, 100000, 100000, 17, 39, 172, 10))


Inserting handle into data store.  output_test_set: inprogress_output_test_set.pq, test_set
Inserting handle into data store.  output_inv_redshift: inprogress_output_inv_redshift.pq, inv_redshift
Inserting handle into data store.  model_inform_PZFlow: inprogress_pzflow.pkl, inform_PZFlow
Inserting handle into data store.  output_lsst_error: inprogress_output_lsst_error.pq, lsst_error


In [ ]:
print(path_lst_1)

["/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='0.33672517538070684'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='0.47006111145019536'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='0.6267686605453491'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='0.8275491118431091'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='1.0106754302978516'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='1.2042927742004392'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='1.4413679003715512'_lsstErr_pzflow.yml", "/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='1.6783331394195558'_ls

In [ ]:
import ceci 

pr = ceci.Pipeline.read(path_lst_1[0])#parent_dir+directory+"/invz=0.33672517538070684_lsstErr_pzflow.yml")
pr.run()

## 1) terminal: go to path up to invz_lsstErr_pzflow, then run these 2 lines 
## 2)  make list/txt file with list of paths to files made by big F

## do 1) 
## open virtual env
## python 
## import ceci 
## run the 2 lines of code above 


### at the end we can put this into a .py file that we can run at the command line 

## %cd ? 


Executing test_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines.flowEngine.FlowCreator   --model=/global/u2/a/acraffor/Photo-z-Stress-Test/data/trained_flow.pkl   --name=test_set   --config=/global/u2/a/acraffor/Photo-z-Stress-Test/Pipeline_Outputs/invz_lsstErr_pzflow/invz='0.337'_lsstErr_pzflow_config.yml   --output=./output_test_set.pq 
Output writing to ./test_set.out

Job test_set has failed with status 1



*************************************************
Error running pipeline stage test_set.

Standard output and error streams in ./test_set.out
*************************************************


1

In [ ]:
## more config parameters/better config parameters
## have to give path above to estimator model instead of get_handle('model')
## fix truncated parameter printing in help(...)

# Big F 2.0

In [39]:
def bigF2(degrader, name, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    testData = testSet(ntest, seed2)
    lsstErr = lsstError(band_dict, seed3)
    infPZFlow = informPZFlow()
    estPZFlow = estimatePZFlow(infPZFlow)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        testData, 
        lsstErr,  
        infPZFlow, 
        estPZFlow]
        #informFZB, 
        #estFZB]

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infPZFlow.connect_input(deg) 
    estPZFlow.connect_input(infPZFlow, inputTag = 'model')
    estPZFlow.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    # informFZB.connect_input(invRed)
    # estFZB.connect_input(informFZB, lsstErr) 

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(path_2, "% s_lsstErr_pzflow.yml" % name)
    pipe.save(outpath)
    return outpath 

In [ ]:
# help(rail.creation.degradation.spectroscopic_selections)

In [50]:
##run 

path_lst_2 = []
directory = "specSelection_lsstErr_pzflow"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_2 = os.path.join(parent_dir, directory)
os.makedirs(path_2, exist_ok=True)

In [51]:
for key in spec_dict:
    path_lst_2.append(bigF2(spec_dict[key], key, 1000, 100, 17, 39, 172, 10))

In [48]:
pr = ceci.Pipeline.read(path_2+"/BOSS_lsstErr_pzflow.yml")
pr.run()



Executing test_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines.flowEngine.FlowCreator   --model=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl   --name=test_set   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_pzflow/BOSS_lsstErr_pzflow_config.yml   --output=./output_test_set.pq 
Output writing to ./test_set.out

Job test_set has completed successfully!

Executing lsst_error
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.degradation.lsst_error_model.LSSTErrorModel   --input=./output_test_set.pq   --name=lsst_error   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_pzflow/BOSS_lsstErr_pzflow_config.yml   --output=./output_lsst_error.pq 
Output writing to ./lsst_error.out

Job lsst_error has completed successfully!

Executing train_set
Command is:
OMP_NUM_THREADS=1   p

0

In [49]:
help(os.makedirs)


Help on function makedirs in module os:

makedirs(name, mode=511, exist_ok=False)
    makedirs(name [, mode=0o777][, exist_ok=False])
    
    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.

